### Textual explanation creation


In this notebook we will:

1️⃣ Load a model


2️⃣ Generate paths from explainable methods like PGPR


3️⃣ Generate textual explanations through templates

**Load libraries**

In [ ]:
from hopwise.quick_start import load_data_and_model
from hopwise.utils.case_study import full_sort_scores
import pandas as pd

**Define model checkpoint 📍**

In [ ]:
checkpoint = "saved/CAFE-Apr-17-2025_02-36-50.pth"

**Define templates 📍**

Consider a path in the form of: `'user', 'relation1', 'entity1', 'relation2', 'entity2', 'relation3', 'product'`

In [ ]:
templates = [
    "{product} is recommend to you because you {relation1} {entity1} also {relation2} by {entity2}",
]

**Load model checkpoint with its configuration**

In [ ]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=checkpoint)

**Load mappings**

In [ ]:
# we map back the entities and users to their original values in the dataset
def pid2item(x): return dataset.id2token(dataset.iid_field, x)
def eid2entity(x): return dataset.id2token(dataset.entity_field, x)
def uid2user(x): return dataset.id2token(dataset.uid_field, x)
def rid2relation(x): return dataset.id2token(dataset.relation_field, x)

**Specify for which users provide explanations 👤**

In [ ]:
#provide explanations for ORIGINAL user 196 and user 186. 
#Consider that the remapped uids are passed to the model, not the ones you provided!!
uid_series = dataset.token2id(dataset.uid_field, ["196", "186"])

**Get explanations from model inference output 🤩**

In [ ]:
# explanations is a pandas dataframe with columns: uid, product, score, explanation (the produced path)
explanations = full_sort_scores(
    uid_series, model, test_data, device=config["device"], explain = True)

**Fill templates 🤔**

In [ ]:
#  fill templates
e_type2mapping = {'U': uid2user, 'E': eid2entity, 'I': pid2item, 'R': rid2relation}

for i, template in enumerate(templates):
    filled_template = list()
    for path in explanations['path']:
        user, relation1, entity1, relation2, entity2, relation3, product = path

        # remap to original entities and items. relations are already mapped back
        user = e_type2mapping[user[0]](int(user[1:]))
        relation  = e_type2mapping[relation1[0]](int(relation1[1:]))
        entity1 = e_type2mapping[entity1[0]](int(entity1[1:]))
        entity2 = e_type2mapping[entity2[0]](int(entity2[1:]))
        
        filled_template.append(template.format(
            user=user,
            relation1=relation1,
            entity1=entity1,
            relation2=relation2,
            entity2=entity2,
            relation3=relation3,
            product=product
            )
        )
    # we create a new column for each template
    explanations[f'template{i}'] = filled_template

**Show textual explanations 🚀**

In [ ]:
explanations